In [11]:
# ============================================================================
# DISPLAY ALL RAW DATA - EVERY ROW AND COLUMN
# ============================================================================

import pandas as pd

# Remove ALL display limits
pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.max_rows', None)         # Show all rows
pd.set_option('display.width', None)            # No width limit
pd.set_option('display.max_colwidth', None)     # Show full column content

# Load the dataset

df = pd.read_csv("/content/dirty_cafe_sales_with_introduced_issues.csv")

# Display complete dataset information
print("="*100)
print("COMPLETE RAW HEALTHCARE DATASET")
print("="*100)
print(f"\nDataset Size: {df.shape[0]} rows × {df.shape[1]} columns")
print("\n")

# This will show EVERY single row and column
print(df.to_string())

Streaming output truncated to the last 5000 lines.
5030     TXN_7160196         Sandwich        1            4.0         4.0             NaN  In-store       2023-12-06
5031     TXN_4634894           Coffee        1            2.0         2.0             NaN       NaN       2023-12-09
5032     TXN_8214944          UNKNOWN        3            1.5         4.5     Credit Card  In-store       2023-11-18
5033     TXN_1080377             Cake        2            3.0         6.0           ERROR       NaN       2023-04-08
5034     TXN_8566683         Smoothie        4            4.0        16.0            Cash  Takeaway       2023-08-09
5035     TXN_2784588            Juice        2            3.0         6.0            Cash  Takeaway       2023-03-26
5036     TXN_6110769           Cookie        2            1.0         2.0             NaN       NaN       2023-03-02
5037     TXN_9999124            Juice        2            3.0         6.0  Digital Wallet  Takeaway          UNKNOWN
5038     TXN_

In [12]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

def print_section(title, char="="):
    """Print formatted section headers"""
    print(f"\n{char*80}")
    print(f" {title}")
    print(f"{char*80}")

def print_subsection(title):
    """Print formatted subsection headers"""
    print(f"\n{title}")
    print("-" * len(title))

# ============================================================================
# 1. LOAD DATA
# ============================================================================
print_section("CAFE SALES DATA ANALYSIS & ASSOCIATION RULE MINING")

df = pd.read_csv("dirty_cafe_sales_with_introduced_issues.csv")

print_section("STEP 1: ORIGINAL DATA INSPECTION")
print(f"\n📊 Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"📋 Columns: {list(df.columns)}")

print_subsection("First 10 Rows")
print(df.head(10))

print_subsection("Data Types")
print(df.dtypes)

print_subsection("Missing Values Summary")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values")

print_subsection("Basic Statistics")
print(df.describe(include='all'))

# ============================================================================
# 2. DATA QUALITY ASSESSMENT
# ============================================================================
print_section("STEP 2: DATA QUALITY ASSESSMENT")

quality_issues = {}

# Check missing values
total_nulls = df.isnull().sum().sum()
quality_issues['missing_values'] = total_nulls
print(f"\n❌ Missing Values: {total_nulls}")
if total_nulls > 0:
    print(df.isnull().sum()[df.isnull().sum() > 0])

# Check duplicates
dup_count = df.duplicated().sum()
quality_issues['duplicates'] = dup_count
print(f"\n❌ Duplicate Rows: {dup_count}")

# Check date format
date_issues = pd.to_datetime(df['Transaction Date'], errors='coerce').isnull().sum()
quality_issues['invalid_dates'] = date_issues
print(f"\n❌ Invalid Dates: {date_issues}")

# Check numeric data
numeric_issues = pd.to_numeric(df['Total Spent'], errors='coerce').isnull().sum()
quality_issues['invalid_numeric'] = numeric_issues
print(f"\n❌ Invalid Numeric Values (Total Spent): {numeric_issues}")

# Check item column
semicolon_items = df['Item'].astype(str).str.contains(';').sum()
comma_items = df['Item'].astype(str).str.contains(',').sum()
blank_items = (df['Item'].astype(str).str.strip() == '').sum()
quality_issues['item_issues'] = semicolon_items + comma_items + blank_items
print(f"\n❌ Item Column Issues:")
print(f"   - Semicolons: {semicolon_items}")
print(f"   - Commas: {comma_items}")
print(f"   - Blank items: {blank_items}")

# Check for unknown/error values
unknown_payment = df['Payment Method'].isin(['UNKNOWN', 'ERROR']).sum()
unknown_location = df['Location'].isin(['UNKNOWN', 'ERROR']).sum()
quality_issues['unknown_values'] = unknown_payment + unknown_location
print(f"\n❌ Unknown/Error Values:")
print(f"   - Payment Method: {unknown_payment}")
print(f"   - Location: {unknown_location}")

print(f"\n📊 Total Issues Found: {sum(quality_issues.values())}")

# ============================================================================
# 3. DATA CLEANING & PREPROCESSING
# ============================================================================
print_section("STEP 3: DATA CLEANING & PREPROCESSING")

df_cleaned = df.copy()
cleaning_log = []

# Clean Transaction Dates
print("\n🔧 [1/6] Cleaning Transaction Dates...")
df_cleaned['Transaction Date'] = pd.to_datetime(df_cleaned['Transaction Date'], errors='coerce')
null_dates = df_cleaned['Transaction Date'].isnull().sum()
df_cleaned['Transaction Date'] = df_cleaned['Transaction Date'].ffill()
cleaning_log.append(f"   ✓ Fixed {null_dates} invalid dates using forward fill")
print(cleaning_log[-1])

# Fix Total Spent
print("\n🔧 [2/6] Fixing Total Spent Column...")
df_cleaned['Total Spent'] = pd.to_numeric(df_cleaned['Total Spent'], errors='coerce')
invalid_spent = df_cleaned['Total Spent'].isnull().sum()
if invalid_spent > 0:
    median_val = df_cleaned['Total Spent'].median()
    df_cleaned['Total Spent'] = df_cleaned['Total Spent'].fillna(median_val)
    cleaning_log.append(f"   ✓ Fixed {invalid_spent} invalid values with median (${median_val:.2f})")
    print(cleaning_log[-1])

# Clean Item Column
print("\n🔧 [3/6] Cleaning Item Column...")
df_cleaned['Item'] = df_cleaned['Item'].astype(str)
df_cleaned['Item'] = df_cleaned['Item'].str.replace(";", "").str.replace(",", "")
df_cleaned['Item'] = df_cleaned['Item'].str.strip().str.lower()
blank_count = (df_cleaned['Item'] == "").sum()
df_cleaned = df_cleaned[df_cleaned['Item'] != ""]
cleaning_log.append(f"   ✓ Removed special characters and {blank_count} blank items")
print(cleaning_log[-1])

# Clean Payment Method
print("\n🔧 [4/6] Cleaning Payment Method...")
invalid_payment = df_cleaned['Payment Method'].isin(['UNKNOWN', 'ERROR']).sum()
df_cleaned['Payment Method'] = df_cleaned['Payment Method'].replace(["UNKNOWN", "ERROR"], np.nan)
df_cleaned['Payment Method'] = df_cleaned['Payment Method'].ffill()
cleaning_log.append(f"   ✓ Fixed {invalid_payment} unknown/error values")
print(cleaning_log[-1])

# Clean Location
print("\n🔧 [5/6] Cleaning Location...")
invalid_location = df_cleaned['Location'].isin(['UNKNOWN', 'ERROR']).sum()
df_cleaned['Location'] = df_cleaned['Location'].replace(["UNKNOWN", "ERROR"], np.nan)
df_cleaned['Location'] = df_cleaned['Location'].ffill()
cleaning_log.append(f"   ✓ Fixed {invalid_location} unknown/error values")
print(cleaning_log[-1])

# Remove Duplicates
print("\n🔧 [6/6] Removing Duplicates...")
dup_count = df_cleaned.duplicated().sum()
df_cleaned = df_cleaned.drop_duplicates()
cleaning_log.append(f"   ✓ Removed {dup_count} duplicate rows")
print(cleaning_log[-1])

# ============================================================================
# 4. CLEANED DATA SUMMARY
# ============================================================================
print_section("STEP 4: CLEANED DATA SUMMARY")

print(f"\n📊 Original Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"📊 Cleaned Shape: {df_cleaned.shape[0]} rows × {df_cleaned.shape[1]} columns")
print(f"📊 Rows Removed: {df.shape[0] - df_cleaned.shape[0]}")
print(f"📊 Data Retention: {(df_cleaned.shape[0]/df.shape[0])*100:.2f}%")

print_subsection("Cleaned Data Sample")
print(df_cleaned.head(10))

print_subsection("Final Data Quality Check")
remaining_nulls = df_cleaned.isnull().sum().sum()
print(f"Missing Values: {remaining_nulls} ✓" if remaining_nulls == 0 else f"Missing Values: {remaining_nulls} ⚠️")
print(f"Duplicates: 0 ✓")

# ============================================================================
# 5. ASSOCIATION RULE MINING PREPARATION
# ============================================================================
print_section("STEP 5: ASSOCIATION RULE MINING")

print_subsection("Preparing Transaction Data")

# Group items by transaction
transactions = df_cleaned.groupby("Transaction ID")["Item"].apply(list).tolist()

print(f"\n📦 Total Transactions: {len(transactions)}")
print(f"📦 Unique Items: {df_cleaned['Item'].nunique()}")
print(f"📦 Avg Items per Transaction: {df_cleaned.groupby('Transaction ID')['Item'].count().mean():.2f}")

print("\n🛒 Sample Transactions:")
for i, trans in enumerate(transactions[:5], 1):
    print(f"   {i}. {trans}")

# One-hot encode transactions
print("\n⚙️ Converting to basket format...")
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_array, columns=te.columns_)

print(f"✓ Basket created: {basket.shape[0]} transactions × {basket.shape[1]} items")

# ============================================================================
# 6. APRIORI ALGORITHM - FREQUENT ITEMSETS
# ============================================================================
print_section("STEP 6: FREQUENT ITEMSETS (APRIORI ALGORITHM)")

min_support = 0.05
print(f"\n⚙️ Minimum Support: {min_support} ({min_support*100}%)")
print("⚙️ Running Apriori algorithm...")

frequent_items = apriori(basket, min_support=min_support, use_colnames=True)
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x: len(x))
frequent_items = frequent_items.sort_values(by='support', ascending=False)

print(f"\n✓ Found {len(frequent_items)} frequent itemsets")

print_subsection("Itemset Distribution")
print(frequent_items['length'].value_counts().sort_index())

print_subsection("Top 15 Frequent Itemsets")
display_items = frequent_items.head(15).copy()
display_items['itemsets'] = display_items['itemsets'].apply(lambda x: ', '.join(list(x)))
print(display_items[['itemsets', 'support', 'length']].to_string(index=False))

# ============================================================================
# 7. ASSOCIATION RULES
# ============================================================================
print_section("STEP 7: ASSOCIATION RULES MINING")

min_confidence = 0.3
print(f"\n⚙️ Minimum Confidence: {min_confidence} ({min_confidence*100}%)")
print("⚙️ Generating association rules...")

rules = association_rules(frequent_items, metric="confidence", min_threshold=min_confidence)

if len(rules) == 0:
    print("\n⚠️ No rules found with current thresholds.")
    print("💡 Try lowering min_support or min_confidence")
else:
    # Format rules
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

    print(f"\n✓ Generated {len(rules)} association rules")

    # Display rules by different metrics
    print_section("ASSOCIATION RULES RESULTS", "=")

    print_subsection("📈 TOP 10 RULES BY LIFT (Strongest Associations)")
    top_lift = rules.nlargest(10, 'lift')[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    print(top_lift.to_string(index=False))

    print_subsection("🎯 TOP 10 RULES BY CONFIDENCE (Most Reliable)")
    top_conf = rules.nlargest(10, 'confidence')[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    print(top_conf.to_string(index=False))

    print_subsection("📊 TOP 10 RULES BY SUPPORT (Most Frequent)")
    top_supp = rules.nlargest(10, 'support')[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    print(top_supp.to_string(index=False))

    # Statistical summary
    print_subsection("📊 Rules Statistics")
    print(f"Average Support:    {rules['support'].mean():.4f}")
    print(f"Average Confidence: {rules['confidence'].mean():.4f}")
    print(f"Average Lift:       {rules['lift'].mean():.4f}")
    print(f"Max Lift:           {rules['lift'].max():.4f}")

    # Key insights
    print_section("KEY INSIGHTS & INTERPRETATION", "=")

    print("""
📖 METRIC DEFINITIONS:
   • Support:     Frequency of itemset in all transactions (popularity)
   • Confidence:  P(consequent|antecedent) - How often rule is correct
   • Lift:        Correlation strength between items

📊 LIFT INTERPRETATION:
   • Lift > 1:  Items are positively correlated (bought together)
   • Lift = 1:  Items are independent (no correlation)
   • Lift < 1:  Items are negatively correlated (rarely together)

💡 BUSINESS RECOMMENDATIONS:
   • High Lift Rules:       Bundle products, cross-selling opportunities
   • High Confidence Rules: Product recommendations, upselling
   • High Support Rules:    Popular combos, promotion strategies
    """)

    # Highlight strongest associations
    strongest = rules.nlargest(3, 'lift')
    print_subsection("🌟 TOP 3 STRONGEST ASSOCIATIONS")
    for idx, row in strongest.iterrows():
        print(f"\n   {row['antecedents']} → {row['consequents']}")
        print(f"   └─ Lift: {row['lift']:.2f}x | Confidence: {row['confidence']:.1%} | Support: {row['support']:.1%}")

print_section("✅ ANALYSIS COMPLETE")
print("\n💾 To export results:")
print("   • frequent_items.to_csv('frequent_items.csv', index=False)")
print("   • rules.to_csv('association_rules.csv', index=False)")
print("   • df_cleaned.to_csv('cleaned_cafe_sales.csv', index=False)")


 CAFE SALES DATA ANALYSIS & ASSOCIATION RULE MINING

 STEP 1: ORIGINAL DATA INSPECTION

📊 Dataset Shape: 10030 rows × 8 columns
📋 Columns: ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date']

First 10 Rows
-------------
  Transaction ID      Item Quantity Price Per Unit Total Spent  \
0    TXN_1961373    Coffee        2            2.0         4.0   
1    TXN_4977031      Cake        4            3.0        12.0   
2    TXN_4271903    Cookie        4            1.0       ERROR   
3    TXN_7034554     Salad        2            5.0        10.0   
4    TXN_3160411    Coffee        2            2.0         4.0   
5    TXN_2602893  Smoothie        5            4.0        20.0   
6    TXN_4433211   UNKNOWN        3            3.0         9.0   
7    TXN_6699534  Sandwich        4            4.0        16.0   
8    TXN_4717867       NaN        5            3.0        15.0   
9    TXN_2064365  Sandwich        5            4.